###1.tensors使用
tensor 类似numpy的ndarray，唯一的区别是Tensor可以在GPU上加速运算。
文档地址：https://pytorch.org/docs/torch

In [2]:
from __future__ import print_function
import torch

#创建空矩阵
x = torch.empty(5,3)
print(x)



tensor([[1.8361e+25, 1.4603e-19, 1.6795e+08],
        [4.7423e+30, 4.7393e+30, 9.5461e-01],
        [4.4377e+27, 1.7975e+19, 4.6894e+27],
        [7.9463e+08, 3.2604e-12, 1.7743e+28],
        [2.0535e-19, 5.9682e-02, 7.0374e+22]])


In [3]:
#随机矩阵
x = torch.rand(4,4)
print(x)

tensor([[0.7497, 0.0216, 0.3694, 0.4083],
        [0.2323, 0.3893, 0.8793, 0.4274],
        [0.8983, 0.7803, 0.5027, 0.2955],
        [0.6102, 0.4692, 0.0510, 0.6673]])


In [5]:
#long型全0矩阵
x = torch.zeros(4,4,dtype=torch.long)
print(x)

tensor([[0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]])
torch.int64


In [9]:
#数组直接构建tensor
x = torch.tensor([1.0,2,3,4])
print(x)

tensor([1., 2., 3., 4.])


从一个已有的tensor构建一个tensor。这些方法会重用原来tensor的特征，例如，数据类型，除非提供新的数据。

In [15]:
x = x.new_ones(4,3,dtype=torch.double) #
print(x)

x = torch.randn_like(x,dtype=torch.float) #修改类型
print(x) #size 跟以前一样

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]], dtype=torch.float64)
tensor([[ 1.8104,  0.6268, -0.2872],
        [ 0.8104, -0.8436,  0.4881],
        [ 0.6030,  0.0870, -0.2614],
        [ 0.1288, -0.7688, -0.0999]])


得到tensor形状

In [16]:
print(x.size())

torch.Size([4, 3])


运算

In [19]:
#加法运算, 两种方式
y = torch.rand(4,3)

print(x+y)

print(torch.add(x,y))

#加法：把输出作为变量

result = torch.empty(4,3)
torch.add(x,y,out= result)
print(result)


tensor([[ 2.5407,  1.2373,  0.6964],
        [ 0.9767, -0.5805,  0.7920],
        [ 1.4973,  0.4174,  0.1200],
        [ 0.4210,  0.1459,  0.3271]])
tensor([[ 2.5407,  1.2373,  0.6964],
        [ 0.9767, -0.5805,  0.7920],
        [ 1.4973,  0.4174,  0.1200],
        [ 0.4210,  0.1459,  0.3271]])
tensor([[ 2.5407,  1.2373,  0.6964],
        [ 0.9767, -0.5805,  0.7920],
        [ 1.4973,  0.4174,  0.1200],
        [ 0.4210,  0.1459,  0.3271]])


in-place加法:y=x+y,结果返回到y上 

In [20]:
y.add_(x)
print(y)

tensor([[ 2.5407,  1.2373,  0.6964],
        [ 0.9767, -0.5805,  0.7920],
        [ 1.4973,  0.4174,  0.1200],
        [ 0.4210,  0.1459,  0.3271]])


注意：
任何in-place的运算都会以``_``结尾。 举例来说：``x.copy_(y)``, ``x.t_()``, 会改变 ``x``。

各种类似NumPy的indexing都可以在PyTorch tensor上面使用。

In [23]:
print(x[:2,1:])

tensor([[ 0.6268, -0.2872],
        [-0.8436,  0.4881]])


Resizing: 如果你希望resize/reshape一个tensor，可以使用torch.view：

In [31]:
x = torch.randn(4, 4)
y = x.view(16)
z = x.view(-1, 8)
print(x.size(), y.size(),z.size())

torch.Size([4, 4]) torch.Size([16]) torch.Size([2, 8])


如果你有一个只有一个元素的tensor，使用.item()方法可以把里面的value变成Python数值。

In [32]:
x = torch.randn(1)
print(x)
print(x.item())

tensor([1.2475])
1.2474795579910278


###numpy和tensor之间转换

Torch Tensor和NumPy array会共享内存，所以改变其中一项也会改变另一项。

把Torch Tensor转变成NumPy Array

In [33]:
a = torch.ones(8)
print(a)

tensor([1., 1., 1., 1., 1., 1., 1., 1.])


In [34]:
b = a.numpy() #转numpy
print(b) 

[1. 1. 1. 1. 1. 1. 1. 1.]


改变numpy array里面的值。

In [35]:
a.add_(2)
print(a)
print(b)

tensor([3., 3., 3., 3., 3., 3., 3., 3.])
[3. 3. 3. 3. 3. 3. 3. 3.]


把NumPy ndarray转成Torch Tensor

In [37]:
import numpy as np
a = np.ones(5)
b = torch.from_numpy(a)
np.add(a,2,out = a)
print (a)
print(b)

[3. 3. 3. 3. 3.]
tensor([3., 3., 3., 3., 3.], dtype=torch.float64)


所有CPU上的Tensor都支持转成numpy或者从numpy转成Tensor。